In [4]:
class NormalClass:
    print("Loading Normal Class")
    spam = "eggs"
    print("Done Loading ")


nco = NormalClass()
print(nco.spam)

Loading Normal Class
Done Loading 
eggs


In [8]:
DynamicClass = type("DyanmicClass", (object,), {"spam": "eggs"})
dco = DynamicClass()
print(dco.spam)

eggs


In [4]:
class Metaclass(type):
    def __init__(cls, name, bases, attrs):
        print("Defining %s" % cls)
        print("Name: %s" % name)
        # print(f"Base: {bases}")
        print("Attrs: %s" % attrs)
        for (name, value) in attrs.items():
            # if name.startswith("__"):
            #     continue
            print("Name: %s" % name, "Value: %s" % value)
        super(Metaclass, cls).__init__(name, bases, attrs)
    
class RealClass(metaclass=Metaclass):
    print("Loading Real Class")
    spam = "eggs"
    
    def fn(self):
        print(self.spam)
    
    print("Done Loading ")


# class SubClass(RealClass):
#     print("Loading Sub Class")
#     spam = "ham"
#     print("Done Loading ")
# 
# sc = SubClass()
# sc.fn()

Loading Real Class
10
Done Loading 
Defining <class '__main__.RealClass'>
Name: RealClass
Attrs: {'__module__': '__main__', '__qualname__': 'RealClass', 'spam': 'eggs', '__init__': <function RealClass.__init__ at 0x7fd38c5c5940>, 'fn': <function RealClass.fn at 0x7fd38c5c6160>, 'f': <function RealClass.<lambda> at 0x7fd38c5c5a80>}
Name: __module__ Value: __main__
Name: __qualname__ Value: RealClass
Name: spam Value: eggs
Name: __init__ Value: <function RealClass.__init__ at 0x7fd38c5c5940>
Name: fn Value: <function RealClass.fn at 0x7fd38c5c6160>
Name: f Value: <function RealClass.<lambda> at 0x7fd38c5c5a80>


In [12]:
class BaseAttribute:
    creation_counter = 1

    def __init__(self):
        self.creation_counter = BaseAttribute.creation_counter
        BaseAttribute.creation_counter += 1

In [14]:
c = BaseAttribute()
d = BaseAttribute()

print(c.creation_counter, d.creation_counter, BaseAttribute.creation_counter)

3 4 5


In [16]:
class Multiplyer(object):
    def __init__(self, factor):
        self.factor = factor
    
    def __call__(self, value):
        return value * self.factor

times2 = Multiplyer(2)
times2(5)
callable(times2)

True

In [1]:
class MyDict(dict):
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            self[key.lower()] = value
    
    def __contains__(self, key):
        return super(MyDict, self).__contains__(key.lower())
    
    def __getitem__(self, __key):
        return super().__getitem__(__key.lower())
    
    def __setitem__(self, __key, __value):
        super().__setitem__(__key.lower(), __value)

d = MyDict(Hello="World", Foo="Bar")
print(d["hello"])
print(d["Hello"])
print(d["HEllo"])

World
World
World


In [32]:
class Fibonacci:
    def __init__(self, count):
        self.count = count
    
    def __iter__(self):
        a, b = 0, 1
        for x in range(self.count):
            if x in [0, 1]:
                yield x
            else:
                c = a + b
                a = b
                b = c
                yield c

for x in Fibonacci(10):
    print(x)

0
1
1
2
3
5
8
13
21
34


In [40]:
class FibIterator:
    def __init__(self, count):
        self.a = 0 
        self.b = 1 
        self.count = count
        self.current = -1
    
    def __next__(self):
        self.current += 1
        if self.current > self.count:
            raise StopIteration
        if self.current in [0, 1]:
            return self.current
        else:
            c = self.a + self.b
            self.a = self.b
            self.b = c
            return c

    next = __next__

class Fibonacci:
    def __init__(self, count):
        self.count = count
    
    def __iter__(self):
        return FibIterator(self.count)

for x in Fibonacci(10):
    print(x)

0
1
1
2
3
5
8
13
21
34
55


In [46]:
class MySeq:
    def __getitem__(self, index):
        if isinstance(index, int):
            return index ** 2
        elif isinstance(index, slice):
            start = index.start or 0
            stop = index.stop
            step = index.step or 1
            return [x ** 2 for x in range(start, stop, step)]

s = MySeq()
for c in range(10):
    print(s[c])

0
4
16
36
64


In [34]:
def decorate(func=None, prefix="Decorated"):
    def decorated(func):
        print("func", func)

        def wrapper(*args, **kwargs):
            print(prefix, func(*args, **kwargs))
        return wrapper
    if func is None:
        print("func inside None", func)
        def decorator(func):
            return decorated(func)
        return decorator
    print(func)
    return decorated(func)

@decorate()
def test(a, b):
    return a+b

# test = decorate(test)

test(1, 2)

func inside None None
func <function test at 0x7fdfdc6ef4c0>
Decorated 3


In [18]:
import functools
test = functools.partial(test, 10)
test(20)
test(400)

ValueValue 30
ValueValueValueValue 410


In [68]:
import datetime

class CurrentDateField:
    def __get__(self, instance, owner):
        if not instance:
            raise AttributeError("Cannot access the current date from the class")
        return datetime.datetime.today()
    def __set__(self, instance, value):
        raise NotImplementedError("Cannot change the current date")


class IntegetField:
    def __init__(self, default=None):
        self.default = default
    
    def __get__(self, instance, owner):
        # get all the instance of the object
        print(instance.__dict__)
        return self.default
    
    def __set__(self, instance, value):
        self.default = value


class Person:
    current_date = CurrentDateField()
    age = IntegetField(default=100)

p = Person()
Person.module = "ArithmeticError"
p.__class__.module

'ArithmeticError'

In [69]:
AC = type('AClass', (object,), {'spam': 'eggs'})
print(AC)
print(type("String"))

<class '__main__.AClass'>
<class 'str'>


In [83]:
import inspect

def get_defaults(func):
    args, varargs, varkw, default, *_ = inspect.getfullargspec(func)
    index = len(args) - len(default)
    return dict(zip(args[index:], default))


def test(a, b, c=10, d=20, *args, **kwargs):
    """
    Useless function
    arg
        The argument to be processed
    """
    pass

get_defaults(test)
test.__doc__ == inspect.getdoc(test)

False

In [85]:
class SubClassTracker(type):
    def __init__(cls, name, bases, attrs):
        print(cls, name, bases, attrs)
        try:
            if TrackedClass not in bases:
                return
        except NameError:
            return 
        TrackedClass._registry.append(cls)

class TrackedClass(metaclass=SubClassTracker):
    _registry = []

class ClassOne(TrackedClass):
    pass 

TrackedClass._registry


<class '__main__.TrackedClass'> TrackedClass () {'__module__': '__main__', '__qualname__': 'TrackedClass', '_registry': []}
<class '__main__.ClassOne'> ClassOne (<class '__main__.TrackedClass'>,) {'__module__': '__main__', '__qualname__': 'ClassOne'}


[__main__.ClassOne]

In [8]:
import heapq

numbers = [1, 4, 2, 100, 20, 50, 32]
four_smallest_nums = heapq.nsmallest(4, numbers)
print(four_smallest_nums)

four_largest = heapq.nlargest(4, numbers)
print(four_largest)

# [1, 2, 4, 20]
# [100, 50, 32, 20]

[1, 2, 4, 20]
[100, 50, 32, 20]
